In [2]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.huggingface.processing import HuggingFaceProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

role = get_execution_role()
language = "en" #change this to de, making sure the data is in german
output_file_name = 'augmented_reviews.csv'

In [3]:
input_data = 's3://sm-data-aug-nlp/inputs/imdb.csv'

In [ ]:
hf_processor = HuggingFaceProcessor(
    role = role, 
    instance_type = 'ml.p3.2xlarge',
    transformers_version = '4.6',
    pytorch_version = '1.7',
    instance_count = 1,
)

In [ ]:
inputs = [ProcessingInput(
    source = input_data, 
    destination = "/opt/ml/processing/input")
         ]
outputs = [ProcessingOutput(
    output_name = 'augemented-text', 
    source="/opt/ml/processing/output", 
    destination ='s3://sm-data-aug-nlp/output/')
          ]
arguments = ["--file-name", "imdb.csv", 
             "--output-file-name", output_file_name,
             "--language", language]

In [ ]:
hf_processor.run(
    code = 'scripts/aug-hf.py',
    inputs = inputs, 
    outputs = outputs, 
    arguments = arguments,
    wait = False
)